# ER131 Land use regression / multiple regression

This notebook does some early explorations of a data set we'll use at a few different points in the course.  See lecture materials for background.

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt

Import the data set:

In [2]:
df_all = pd.read_csv('BechleLUR_2006_allmodelbuildingdata.csv')

Let's see what's inside:

In [3]:
for i in df_all.columns:
    print(i,end="\n ")

Monitor_ID
 State
 Latitude
 Longitude
 Observed_NO2_ppb
 Predicted_NO2_ppb
 WRF+DOMINO
 Distance_to_coast_km
 Elevation_truncated_km
 Impervious_100
 Impervious_200
 Impervious_300
 Impervious_400
 Impervious_500
 Impervious_600
 Impervious_700
 Impervious_800
 Impervious_1000
 Impervious_1200
 Impervious_1500
 Impervious_1800
 Impervious_2000
 Impervious_2500
 Impervious_3000
 Impervious_3500
 Impervious_4000
 Impervious_5000
 Impervious_6000
 Impervious_7000
 Impervious_8000
 Impervious_10000
 Population_100
 Population_200
 Population_300
 Population_400
 Population_500
 Population_600
 Population_700
 Population_800
 Population_1000
 Population_1200
 Population_1500
 Population_1800
 Population_2000
 Population_2500
 Population_3000
 Population_3500
 Population_4000
 Population_5000
 Population_6000
 Population_7000
 Population_8000
 Population_10000
 Major_100
 Major_200
 Major_300
 Major_400
 Major_500
 Major_600
 Major_700
 Major_800
 Major_1000
 Major_1200
 Major_1500
 Major_1

In [4]:
df_all.head()

,Monitor_ID,State,Latitude,Longitude,Observed_NO2_ppb,Predicted_NO2_ppb,WRF+DOMINO,Distance_to_coast_km,Elevation_truncated_km,Impervious_100,...,total_8000,total_10000,total_10500,total_11000,total_11500,total_12000,total_12500,total_13000,total_13500,total_14000
0,04-013-0019-42602-1,AZ,33.48385,-112.14257,23.884706,20.986643,11.615223,313.0,0.304,59.4431,...,1788.38015,2637.91917,2862.73591,3096.99468,3339.22952,3609.20650,3896.25748,4150.54739,4396.96011,4651.18890
1,04-013-3002-42602-6,AZ,33.45793,-112.04601,25.089886,20.990096,11.472677,323.8,0.304,72.0000,...,1731.04787,2562.32948,2791.32295,3015.79024,3248.95785,3489.76919,3723.01595,3963.41655,4196.37496,4459.57421
2,04-013-3003-42602-1,AZ,33.47968,-111.91721,19.281969,18.088153,8.990372,308.4,0.304,53.0000,...,1254.14847,1965.43346,2157.42878,2362.96458,2584.38952,2820.52494,3052.44507,3315.05126,3607.37536,3921.12841
3,04-013-3010-42602-1,AZ,33.46093,-112.11748,30.645138,20.358009,11.919268,309.0,0.304,61.3099,...,1599.66889,2449.51041,2660.60636,2879.53599,3109.74604,3339.37790,3597.15279,3848.61451,4125.08884,4427.95530
4,04-013-4011-42602-1,AZ,33.37005,-112.62070,11.070412,8.549622,2.141366,269.5,0.293,12.0000,...,149.29461,222.34687,244.41106,269.54740,293.31410,320.37722,349.76462,386.03419,412.91888,441.52860


Question: Which variables would be a bad idea to include in the model?  That is, before we try to do any model selection, what's obviously out?

Now let's build some models.

First let's try regressing observed NO2 against the satellite measuremenets (you'll do this in HW)

In [5]:
lm = linear_model.LinearRegression(fit_intercept=True)

Very important note: scikit-learn wants a data frame for the predictors -- if you have only one predictor and you pass in a pandas series, scikit-learn throws an error. 

In [6]:
predictors = df_all.loc[:,['WRF+DOMINO']]
output = df_all['Observed_NO2_ppb']
lm.fit(predictors, output)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
print('slope is',lm.coef_[0])
print('intercept is',lm.intercept_)

In [ ]:
y_hat = lm.predict(predictors)

In [ ]:
plt.scatter(output,y_hat)
plt.xlabel('Observed NO2')
plt.ylabel('Predicted NO2')

In [ ]:
plt.scatter(predictors, output-y_hat)
plt.xlabel('Satellite NO2')
plt.ylabel('Residual')

Now let's look at the regression results using a different package called statsmodels

In [ ]:
import statsmodels.api as sm
from scipy import stats

There is a nice feature in statsmodels that allows you to add a constant to a dataframe:

In [ ]:
predictors_const = sm.add_constant(predictors)
predictors_const.head()

In [ ]:
est = sm.OLS(output, predictors_const)
est_fit = est.fit()
print(est_fit.summary())

Now let's try estimating a model with **all** the predictors embedded:

In [ ]:
predictors_all = df_all.loc[:,'WRF+DOMINO':'total_14000']
predictors_all_const = sm.add_constant(predictors_all)
est_all = sm.OLS(output, predictors_all_const)
est_all_fit = est_all.fit()
print(est_all_fit.summary())

Now let's look at what happens if we drop some of the predictors

In [ ]:
predictors_less = df_all.loc[:,'WRF+DOMINO':'Impervious_100']
predictors_less_const = sm.add_constant(predictors_less)
est_less = sm.OLS(output, predictors_less_const)
est_less_fit = est_less.fit()
print(est_less_fit.summary())

Some things to note:
1. AIC is lower for \*\_less.  \*\_all does a better job of reducing squared error, but it gets penalized more for having more variables.
2. Look at the p-values and confidence intervals.  You can see that p-values are big (>0.05) when confidence intervals span zero.  
3. Resident_100 was insignificant when included with lots of other Resident_\* variables, but on it's own it is strongly significant.   